In [1]:
import polars as pl
import polars_ols as pls
import numpy as np

In [2]:
def _make_data(n_samples: int = 2_000, 
               n_features: int = 5,
               n_groups: int = 5,
               noise: float = 0.1,
              ) -> pl.DataFrame:
    x = np.random.normal(size=(n_samples, n_features))
    eps = np.random.normal(size=n_samples, scale=noise)
    return pl.DataFrame(data=x, schema=[f"x{i + 1}" for i in range(n_features)]).with_columns(
        y=pl.lit(-1 * x.sum(1) + eps),
        group=pl.lit(np.random.randint(0, n_groups, size=n_samples)),
        sample_weights=pl.lit(np.random.rand(n_samples)),
    )

In [3]:
df = _make_data(n_samples=2_000, n_features=3, n_groups=5)

In [4]:
df

x1,x2,x3,y,group,sample_weights
f64,f64,f64,f64,i64,f64
0.244378,0.29603,-1.591151,1.057149,3,0.350132
2.093035,-1.194226,1.174871,-2.008692,4,0.589495
0.26513,0.254817,1.908368,-2.423658,0,0.308177
-0.139459,0.201641,-0.750208,0.698626,1,0.378428
0.230555,1.111359,0.229246,-1.635799,1,0.138395
…,…,…,…,…,…
1.209234,-0.792829,0.277771,-0.895549,2,0.965688
-0.172662,-2.526893,-0.527852,3.20544,2,0.569951
1.835942,0.900255,1.29236,-4.14354,1,0.680204


### 1. Basic Usage: OLS / WLS
- You can use `pls.compute_least_squares` or `least_squares.ols` from the registered namespace. They are equivalent.
- Simply pass an expression producing strictly positive sample weights to `sample_weights` argument to perform WLS

In [5]:
ols_expr = pls.compute_least_squares(pl.col("y"),  # target
                          pl.col("x1"), pl.col("x2"), pl.col("x3"),  # features
                          mode="predictions",
                          )
assert str(ols_expr) == str(pl.col("y").least_squares.ols(pl.col("x1"), pl.col("x2"), pl.col("x3")))

wls_expr = pl.col("y").least_squares.wls(pl.col("x1"), pl.col("x2"), pl.col("x3"), 
                                         sample_weights=pl.col("sample_weights"))

- The expressions returned are normal polars expressions. You can operate on them lazily, so for example we can compute OLS per group in parallel using `.over(...)` or multiply it by some other expression etc.

In [6]:
df.lazy().with_columns(ols_expr.over("group").alias("predictions_ols_group"),
                ols_expr.alias("predictions_ols"),
                (wls_expr * (pl.col("group") == 2)).alias("predictions_wls_masked"),
               ).collect().tail(10)

x1,x2,x3,y,group,sample_weights,predictions_ols_group,predictions_ols,predictions_wls_masked
f64,f64,f64,f64,i64,f64,f32,f32,f32
1.113039,0.883359,-2.527527,0.55024,2,0.699654,0.527899,0.534972,0.531683
-0.162361,-0.31886,0.432553,0.116105,3,0.201684,0.050201,0.047775,0.0
0.513537,0.290477,1.275113,-2.125898,0,0.250123,-2.087187,-2.080451,-0.0
-0.409181,0.795874,-1.508068,1.218333,1,0.252079,1.120106,1.1244,0.0
-1.503989,-1.071248,-1.820427,4.540995,3,0.844558,4.396885,4.396938,0.0
1.209234,-0.792829,0.277771,-0.895549,2,0.965688,-0.688014,-0.695993,-0.698428
-0.172662,-2.526893,-0.527852,3.20544,2,0.569951,3.232339,3.224712,3.225684
1.835942,0.900255,1.29236,-4.14354,1,0.680204,-4.027004,-4.029541,-0.0
-1.14397,0.672722,-0.133538,0.618378,3,0.438267,0.605977,0.606245,0.0


- The `mode` parameter controls the type of output produced. You can choose from {`predictions`, `coefficients`, `residuals`}. It defaults to `predictions`.
- `coefficients` produces a compact struct with the names of your features as fields and estimated coefficients as values

In [7]:
df.select(pl.col("y").least_squares.ols(pl.col("x1"), pl.col("x2"), add_intercept=True, mode="coefficients")
          .alias("coefficients"))

coefficients
struct[3]
"{-0.999718,-1.029042,-0.013478}"


- If in a `.over()`, `.group_by()`, or a `.with_columns()` context, the output of `mode="coefficients"` broadcasts to the shape of your data
- Computing least_squares operations in `.over()` is done in parallel in rust, so it is very efficient
- You can use `.unnest()` to unpack the coefficients to separate numeric columns

In [8]:
df_coefficients = df.select("group", pl.col("y").least_squares.ols(
    pl.col("x1"), pl.col("x2"), add_intercept=True, mode="coefficients").over("group")
          .alias("coefficients"))
print(df_coefficients.head())
print(df_coefficients.unnest("coefficients").head())

shape: (5, 2)
┌───────┬─────────────────────────────────┐
│ group ┆ coefficients                    │
│ ---   ┆ ---                             │
│ i64   ┆ struct[3]                       │
╞═══════╪═════════════════════════════════╡
│ 3     ┆ {-0.995489,-0.984824,-0.01902}  │
│ 4     ┆ {-1.040719,-0.995801,-0.026311} │
│ 0     ┆ {-0.9619,-1.095804,-0.007966}   │
│ 1     ┆ {-0.96268,-1.037212,-0.027022}  │
│ 1     ┆ {-0.96268,-1.037212,-0.027022}  │
└───────┴─────────────────────────────────┘
shape: (5, 4)
┌───────┬───────────┬───────────┬───────────┐
│ group ┆ x1        ┆ x2        ┆ const     │
│ ---   ┆ ---       ┆ ---       ┆ ---       │
│ i64   ┆ f32       ┆ f32       ┆ f32       │
╞═══════╪═══════════╪═══════════╪═══════════╡
│ 3     ┆ -0.995489 ┆ -0.984824 ┆ -0.01902  │
│ 4     ┆ -1.040719 ┆ -0.995801 ┆ -0.026311 │
│ 0     ┆ -0.9619   ┆ -1.095804 ┆ -0.007966 │
│ 1     ┆ -0.96268  ┆ -1.037212 ┆ -0.027022 │
│ 1     ┆ -0.96268  ┆ -1.037212 ┆ -0.027022 │
└───────┴───────────┴───────

### 2. Regularized Models
- Ridge `least_squares.ridge`, Lasso `least_squares.lasso`, Elastic Net `least_squares.lasso` with optional non-negative constraint are implemented
- Apart from ridge, which is solved in closed form, the rust implementation for regularized models is cyclic coordinate descent with a soft thresholding function that supports an arbitrary combination of L1 / L2 penalties and non-negative constraint.
- `sample_weights` and `mode` are general parameters applicable to all models supported by this package

Parameters specific to regularized models are contained in `OLSKwargs`:
- alpha: scalar representing L1 or L2 penalty strength.
- l1_ratio: mixing parameter for ElasticNet regularization (0 for Ridge, 1 for LASSO).
- max_iter: maximum number of coordinate descent iterations
- tol: tolerance for convergence criterion
- positive: boolean enforcing non-negativity constraints on coefficients

In [9]:
# inspect OLS Kwargs
pls.OLSKwargs?

Init signature:
pls.OLSKwargs(
    alpha: 'Optional[float]' = 0.0,
    l1_ratio: 'Optional[float]' = None,
    max_iter: 'Optional[int]' = 1000,
    tol: 'Optional[float]' = 1e-05,
    positive: 'Optional[bool]' = False,
    null_policy: 'NullPolicy' = 'ignore',
    solve_method: 'Optional[SolveMethod]' = None,
) -> None
Docstring:     
Specifies parameters relevant for regularized linear models: LASSO / Ridge / ElasticNet.

Attributes:
    alpha: Regularization strength. Defaults to 0.0.
    l1_ratio: Mixing parameter for ElasticNet regularization (0 for Ridge, 1 for LASSO).
        Defaults to None (equivalent to Ridge regression).
    max_iter: Maximum number of iterations. Defaults to 1000 iterations.
    tol: Tolerance for convergence criterion. Defaults to 1.e-5.
    positive: Whether to enforce non-negativity constraints on coefficients.
        Defaults to False (no constraint on coefficients).
    null_policy: Strategy for handling missing data. Defaults to "ignore".
    solve

In [10]:
elastic_net_expr = pl.col("y").least_squares.elastic_net(pl.col("x1"), pl.col("x2"), pl.col("x3"),
                                                         alpha=0.0001,
                                                         l1_ratio=0.5,
                                                         positive=True,
                                                         mode="coefficients",
                                                         ).alias("coef_enet_non_negative")

ridge_expr = pl.col("y").least_squares.ridge(pl.col("x1"), pl.col("x2"), pl.col("x3"),
                                             alpha=100.0, 
                                             sample_weights=pl.col("sample_weights"),
                                             mode="coefficients").alias("coef_ridge")

df.select(elastic_net_expr, ridge_expr)

coef_enet_non_negative,coef_ridge
struct[3],struct[3]
"{0.0,0.0,0.0}","{-0.91927,-0.911867,-0.920933}"


### 3. Formula API

- For those who like specifying models in patsy formula syntax, that is also supported
- You can either use the `least_squares_from_formula` module level public function or `least_squares.from_formula` from registed namespace
- It tries to be clever and maps to the correct underlying implementation based on the model specific parameters you specify

In [12]:
# compute the residuals in two equivalent ways
df.select(
    # "x2:x3" denotes multiplicative interaction, "-1" dentotes no intercept
    pls.compute_least_squares_from_formula("y ~ x1 + x2:x3 -1", mode="residuals").alias("residuals_1"), 
    (pl.col("y") - pl.col("y").least_squares.from_formula("x1 + x2:x3 -1", mode="predictions")).alias("residuals_2"),
).corr()

residuals_1,residuals_2
f64,f64
1.0,1.0
1.0,1.0


In [13]:
nnls_formula_expr = pl.col("y").least_squares.from_formula("x1 + x2 + x3",
                                       alpha=0.0001,
                                       positive=True,
                                       )  # knows to use the coordinate descent implementation because of non-negativity


ridge_formula_expr = pl.col("y").least_squares.from_formula("x1 + x2 + x3",
                                       alpha=0.0001,
                                       sample_weights=pl.col("sample_weights"),
                                       )  # knows that it needs to use closed form ridge w/ sample weighting

### 4. Dynamic Regression Models

- Consider the situation where you want to compute coefficients in an expanding or rolling window manner
    - naively, you could manually re-compute standard OLS function over consecutive windows (e.g. `.rolling(...).agg(...)`)
    - ... but that would be wasteful: (X.T X) and (X.T Y) are only changing by one row (in case of expanding) or two rows (in case of rolling, an addition and a subtraction)
- This extension package provides rust implementations `.least_squares.{rolling_ols, expanding_ols, rls}` which efficiently update coefficients as new samples are observed
- The key idea is to make use of Sherman-Morrison or Woodbury Identity to recursively update summary statistics or coefficient vectors
- Formula API is also supported and the correct implementation is chosen based on parameters provided

In [14]:
df.select(
    pl.col("y").least_squares.from_formula("x1 + x2 + x3 -1", 
                                           window_size=252, 
                                           min_periods=5, 
                                           alpha=0.0001,  
                                           mode="coefficients").over("group").alias("rolling_ridge_coef"),
    pl.col("y").least_squares.rls(
        pl.col("x1"), pl.col("x2"), pl.col("x3"),
        half_life=21.0, # exponential memory proportional to a half-life of 21 samples
        initial_state_mean=[-1.0, -1.0, -1.0],  # prior mean for initial coefficients
        initial_state_covariance=10.0,  # inversely proportional to L2 prior towards prior mean
        mode="coefficients",
    ).over("group").alias("recursive_least_squares_coef"),
    pl.col("y").least_squares.expanding_ols(pl.col("x1"), pl.col("x2"), pl.col("x3"), 
                                           mode="predictions").alias("expanding_ols_pred"),
)

rolling_ridge_coef,recursive_least_squares_coef,expanding_ols_pred
struct[3],struct[3],f32
"{0.0,0.0,0.0}","{-0.999436,-0.999317,-1.003672}",1.01911
"{0.0,0.0,0.0}","{-0.981326,-1.010655,-0.989518}",-1.986911
"{0.0,0.0,0.0}","{-0.999681,-0.999694,-0.997705}",-2.273605
"{0.0,0.0,0.0}","{-1.002054,-0.99703,-1.011049}",0.713363
"{0.0,0.0,0.0}","{-1.01267,-1.047104,-1.022372}",-1.524843
…,…,…
"{-0.99819,-0.998837,-0.99271}","{-1.004297,-1.004904,-0.990937}",-0.695928
"{-0.99821,-0.998581,-0.992718}","{-1.004473,-1.002611,-0.990368}",3.224019
"{-1.004952,-1.003536,-0.99391}","{-1.003174,-0.991553,-1.012009}",-4.029209


### 5. Out Of Sample Prediction

- If you want to fit on some data then predict on test data, you can do so with `least_squares.predict(...)`

In [15]:
# make some random training data
df_train = _make_data(n_groups=1)

# fit coefficients
df_coefficients = (
    df.lazy()
    .select(
        "group",
        pl.col("y")
        .least_squares.ols(pl.col("x1"), pl.col("x2"), mode="coefficients")
        .over("group").alias("coefficients"),
    )
    .unique()
)

df_coefficients.collect()

group,coefficients
i64,struct[2]
2,"{-1.032807,-1.043651}"
3,"{-0.99592,-0.985827}"
4,"{-1.040148,-0.995546}"
0,"{-0.962558,-1.095777}"
1,"{-0.961928,-1.036407}"


In [16]:
# make some test data
df_test = _make_data(n_groups=1)

# 1) join on group or common index columns etc.
# 2) compute predictions by calling least_squares.predict(coefficient_column, *feature_columns)
predictions = (
    df_test.lazy()
    .join(df_coefficients, on="group")
    .select(
        "group",
        pl.col("coefficients").least_squares.predict(
            pl.col("x1"), pl.col("x2"), name="predictions_test"
        )
    )
    .collect()
)

predictions.head()

group,predictions_test
i64,f32
0,5.21625
0,-0.810267
0,0.377538
0,-0.982739
0,-1.07014
